In [1]:
# load necessary libraries. install what you don't have
import numpy as np
import pandas as pd
import requests
import requests
import urllib
import os
import pprint
import json
import matplotlib.pyplot as plt
import seaborn as sns
import nltk 
import sklearn 

In [18]:
df = pd.read_excel("../DATA/original_data.xls", sheet_name="Data")

In [19]:
#Columns from original dataset we want to keep
columns_to_keep = ["Date", "S&P Comp. P", "Earnings", "Real Total Return Price" ]
df = df[columns_to_keep]

#Date format is inconsistent so lets make the formatting better and recognized as a date data type
df["Date"] = pd.to_datetime(df["Date"].astype(str), format="%Y-%m-%d")

#Filter out all dates before 1965, reconsider if this is a good selection later
df = df[df["Date"] >= "1965-01-01"]

#Added PE ratio column
df["PE Ratio"] = df["S&P Comp. P"] / df["Earnings"]

#Drop price and earnings columns because that data is now stored in the PE column
df = df.drop(columns=["S&P Comp. P", "Earnings"])

#df.rename(columns={"10 Year Annualized Stock Real Return": "10 Year Return"}, inplace=True)


print(df.head())

           Date  Real Total Return Price   PE Ratio
1128 1965-01-01            109618.976259  18.748925
1129 1965-02-01            110689.595261  18.709548
1130 1965-03-01            110707.980510  18.553419
1131 1965-04-01            112076.467028  18.585224
1132 1965-05-01            114020.425301  18.651798


In [20]:
# General function to calculate N-year returns
#Takes in 3 paramaters, a date, the dataframe, and then how many years out you want to calculate annual returns for
def get_n_year_return(date, df, years):
    #Calculates a target date by adding desired amount of years to current date to get futrue date (defaults to 10)
    target_date = date + pd.DateOffset(years=years)
    #Filters dataframe to only include dates that occur after the date provided to make searching quicker, then looks for the target date n years out
    
    # Filter the DataFrame to only include dates on or after the target date
    future_rows = df[df["Date"] >= target_date]

    # Check if there are any rows left after filtering
    if not future_rows.empty:
        # Select the first row (earliest available future date)
        future_row = future_rows.iloc[0]
    else:
        # If no future row is found, return None
        future_row = None


    if future_row is not None:
        #If some observation n years out exists, then using the compound annual growth formula given by original dataset calcualte the CAGR annual return 
        return (future_row["Real Total Return Price"] / df[df["Date"] == date]["Real Total Return Price"].values[0]) ** (1/years) - 1
    return None  # Return None if no valid future date is found

# Create an empty list to store results
five_year_returns = []
three_year_returns = []

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    five_year_returns.append(get_n_year_return(row["Date"], df, 5))
    three_year_returns.append(get_n_year_return(row["Date"], df, 3))

# Assign results to new columns
df["5 Year Annual Return"] = five_year_returns
df["3 Year Annual Return"] = three_year_returns

# Display the first few rows
print(df.head())


           Date  Real Total Return Price   PE Ratio  5 Year Annual Return  \
1128 1965-01-01            109618.976259  18.748925              0.002613   
1129 1965-02-01            110689.595261  18.709548             -0.006861   
1130 1965-03-01            110707.980510  18.553419             -0.003975   
1131 1965-04-01            112076.467028  18.585224             -0.013485   
1132 1965-05-01            114020.425301  18.651798             -0.040447   

      3 Year Annual Return  
1128              0.035407  
1129              0.016211  
1130              0.009866  
1131              0.029799  
1132              0.031574  
